## Imports

In [1]:
import numpy as np
import torch
import torch.nn as nn
from torch import optim
import torch.nn.functional as F
from torch.utils.data import DataLoader

from sklearn.manifold import TSNE
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt

from utils import extract_time
from utils import random_generator
from sklearn.preprocessing import MinMaxScaler



from model_train import TimeGAN
from model_train import Time_GAN_module
import pandas as pd


In [2]:
torch.cuda.is_available()

True

## Set Parameters

In [6]:
#output_size = 20
gamma = 1
no, seq_len, dim = 350, 200, 5

parameters = dict()
parameters['hidden_dim'] = 24
parameters['num_layers'] = 3
parameters['iterations'] = 100
parameters['batch_size'] = 10
parameters['epoch'] = 10 #Change the number of epochs as needed

In [7]:
data = pd.read_csv("main_fligh_necessary.csv")
requred_features = ["altitude", "distance", "latitude", "longitude", "groundspeed"]
data = data[requred_features].to_numpy()
scaler = MinMaxScaler()

data = scaler.fit_transform(data)

data = data.reshape([-1,200,len(requred_features)])


data = torch.Tensor(data)
data.shape

torch.Size([350, 200, 5])

### Training and Examination of generating process: for each epoch a single sample is generated from the current state of the generator and visualized alongside with the coressponding loss values

In [8]:
Generator, Embedder, Supervisor, Recovery, Discriminator = TimeGAN(data, parameters)

Start Embedding Network Training
Embedding Loss:tensor(2.2754, device='cuda:0', grad_fn=<MulBackward0>)
step: 1/10
step: 2/10
step: 3/10
step: 4/10
step: 5/10
step: 6/10
step: 7/10
step: 8/10
step: 9/10
Finish Embedding Network Training
Start Training with Supervised Loss Only
Supervisor Loss:tensor(0.0767, device='cuda:0', grad_fn=<MseLossBackward0>)
step: 1/10
step: 2/10
step: 3/10
step: 4/10
step: 5/10
step: 6/10
step: 7/10
step: 8/10
step: 9/10
Finish Training with Supervised Loss Only
Start Joint Training
step: 1/10
step: 2/10
step: 3/10
step: 4/10
step: 5/10
step: 6/10
step: 7/10
step: 8/10
step: 9/10
step: 10/10
Finish Joint Training


### A Data Set of the same size and dimensions of the original synthetic data Set is generated

In [9]:
num_samples = 350
sample =  random_generator(num_samples, dim, [seq_len]*num_samples,seq_len)
test_sample = Generator(sample)[0]
test_sample = torch.reshape(test_sample, (num_samples, seq_len, parameters['hidden_dim']))
test_recovery = Recovery(test_sample)
test_recovery = torch.reshape(test_recovery[0], (num_samples, seq_len, dim))

In [10]:
values = scaler.inverse_transform(test_recovery.cpu().detach().numpy().reshape([-1,len(requred_features)]))

In [11]:
values.shape

(70000, 5)

In [12]:
synthetic_data = pd.DataFrame(values)

In [13]:
synthetic_data.rename(columns={
    0: 'altitude',
    1: 'groundspeed',
    2: 'latitude',
    3: 'longitude',
    4: 'distance'
}, inplace=True)

In [14]:
synthetic_data['flight_ID'] = ['A_{:02d}'.format(i // 200 + 1) for i in range(len(synthetic_data))]

In [15]:
synthetic_data

,altitude,groundspeed,latitude,longitude,distance,flight_ID
0,18942.843750,33.877686,47.635162,8.692516,324.121918,A_01
1,17832.949219,33.897392,47.666931,8.694017,333.232880,A_01
2,17545.914062,33.827274,47.672535,8.694969,335.400848,A_01
3,17469.296875,33.731167,47.670116,8.698079,335.388763,A_01
4,17393.250000,33.582184,47.666271,8.701200,334.566772,A_01
...,...,...,...,...,...,...
69995,4090.293457,7.748353,47.249107,8.773554,167.448303,A_350
69996,4074.410400,7.713489,47.248398,8.773480,167.186768,A_350
69997,4058.984131,7.679347,47.247707,8.773414,166.932053,A_350
69998,4043.991943,7.645894,47.247036,8.773352,166.683868,A_350
